In [2]:
# Heavily adapted from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html and https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

import sys
import os
main_folder_path = "./" 
import torch
import torch.nn as nn
import datetime as dt
import argparse
import subprocess
import ray.cloudpickle as pickle
import numpy as np
import tempfile
import pandas as pd

from utils.dataset import VideoDataset
from ray import tune, train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
from models.rcnn.rcnn import RecurrentConvolutionalNetwork
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from pathlib import Path
from functools import partial
from utils.types import NonlinearityEnum
from torchvision.transforms import v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME = "rcnn"
NOW = dt.datetime.now()
FILENAME = f"{NOW.strftime('%Y-%m-%d-%H-%M-%S')}"
SAVE_DIR = f"{main_folder_path}/models/rcnn/saved_models"
DATA_FOLDER = "data"
INF = 100000000.
NUM_WORKERS = 8
NUM_CLASSES = 2
GPUS_PER_TRIAL = torch.cuda.device_count() if torch.cuda.is_available() else 0

timestamp = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

2024-11-13 21:20:31.332959: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 21:20:31.340877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731504031.350324   16192 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731504031.353149   16192 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:20:31.363596: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [7]:
search_space = {
    "input_channels": 3,
    "num_recurrent_layers": 2,
    "num_kernels": 3,
    "kernel_size": 9,
    "stride": 1,
    "padding": "same",
    "dropout_prob": 0.185931,
    "nonlinearity": NonlinearityEnum.ELU,
    "bias": False,
    "steps": 32,
    "num_classes": NUM_CLASSES,
    "batch_size": 2,
    "lr": 0.0001236517,
    "include_additional_transforms": False
}

In [8]:

best_trained_model = RecurrentConvolutionalNetwork(
    input_channels=search_space["input_channels"],
    num_recurrent_layers=search_space["num_recurrent_layers"],
    num_kernels=search_space["num_kernels"],
    kernel_size=search_space["kernel_size"],
    stride=search_space["stride"],
    padding=search_space["padding"],
    dropout_prob=search_space["dropout_prob"],
    nonlinearity=search_space["nonlinearity"],
    bias=search_space["bias"],
    steps=search_space["steps"],
    num_classes=search_space["num_classes"],
)

In [6]:
from torchviz import make_dot

In [9]:
torch_input = torch.randn(1, 32, 3, 224, 224)
y = best_trained_model(torch_input)

In [ ]:
make_dot(y, params=dict(best_trained_model.named_parameters()), show_attrs=False, show_saved=False).render("models/rcnn/diagrams/rcnn_diagram", format="png")

'models/rcnn/rcnn_diagram.png'